# Xgboost

* similar to Gradient Boosting but more efficient. it has the capacity to do pararerel computation.
* Supprot various objective functions , including regression , classification .

- Boosting is a kind of learning algorithms that try to fit the data by using multiple simpler models, or so called base learner/weak learner. The way it does is to adaptively fit the data by using the same or slightly different in parameter setting’s base learners.

- The very first development is AdaBoost by Freund and Schapire (1996). AdaBoost is actually minimizing the exponential loss function and iteratively get the weak classifier to be trained on weighted data. New boosting algorithm was also proposed which tries to minimize a second-order approximation of the log-loss: LogitBoost.


* XGBoost (eXtreme Gradient Boosting) is a direct application of Gradient Boosting for decision trees. There are a myriad of resources that dive into the mathematical backing and systematic functions of XGBoost, but the main advantages are as follows:

1. Easy to use
2. Computational efficiency
3. Model Accuracy
4. Feasibility — easy to tune parameters and modify objectives.


In [87]:
# Install xgboost follow the instructions on  http://xgboost.readthedocs.io/en/latest/build.html#


# Importing libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [88]:
# Importing the dataset
dataset = pd.read_csv('../Resource/Churn_Modelling.csv')
X = dataset.iloc[:, 3:13]
y = dataset.iloc[:, 13].values

print(dataset.shape)

dataset.head()

(10000, 14)


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [89]:
X.head(2)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58


In [90]:

X=pd.get_dummies(X, columns=['Gender','Geography'], drop_first=True)

X.head(2)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender_Male,Geography_Germany,Geography_Spain
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1


In [91]:
# Splitting the dataset into the Training set and Test set

from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test= train_test_split(X,y , test_size=0.2 , random_state=0)

X_train.shape , X_test.shape , y_train.shape

((8000, 11), (2000, 11), (8000,))

In [107]:
# Fitting XGBoost to the Training set
from sklearn import preprocessing

from xgboost import XGBClassifier

XGB_Model=XGBClassifier(max_depth=5, learning_rate=0.04)

# checking params
XGB_Model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.04, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [108]:
%time

XGB_Model.fit(X_train, y_train)

print ('\n','score :', XGB_Model.score(X_test, y_test))

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 11.9 µs

 score : 0.872


In [109]:
%time

# Cross Validation

from sklearn.model_selection import cross_val_score

cross=cross_val_score(XGB_Model, X_train, y_train, cv=10 )

print ('mean',cross.mean(),'.\n')

print ('std',cross.std())

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 8.82 µs
mean 0.8627496007806263 .

std 0.007462616552096544


In [110]:
%time

# Tunning model with GridSearch 

from sklearn.model_selection import GridSearchCV

from xgboost import XGBClassifier


XG=XGBClassifier()

params={'max_depth':[2,3,4,5,6,7], 'learning_rate':[0.1, 0.2, 0.04, 0.05],'n_estimators':[100, 150]}




GS= GridSearchCV(XG, params,cv=5, n_jobs=-1 )

GS.fit(X_train , y_train)

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 9.06 µs


GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_depth': [2, 3, 4, 5, 6, 7], 'learning_rate': [0.1, 0.2, 0.04, 0.05], 'n_estimators': [100, 150]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [111]:
GS.best_params_

{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}

In [112]:
GS.best_score_

0.864125

In [113]:
GS.score(X_test, y_test)

0.8645

In [99]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix

y_pred=XGB_Model.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
cm

array([[1532,   63],
       [ 193,  212]])

# Compare to Random Forrest Model

In [106]:
%timeit

# check Random Forrest 

from sklearn.ensemble import RandomForestClassifier

RF_Model=RandomForestClassifier(n_estimators=100, n_jobs=-1, max_features=0.05, min_samples_leaf=2)

RF_Model.fit(X_train, y_train)

RF_Model.score(X_test, y_test)

0.864

In [101]:
RF_Model

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=0.05, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=2,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [105]:
%timeit
# Cross Validation

from sklearn.model_selection import cross_val_score

cross=cross_val_score(RF_Model, X_train, y_train, cv=10 )

print ('mean',cross.mean(),'.\n')

print ('std',cross.std())

mean 0.8558747529293015 .

std 0.00767000666106296


In [103]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix

y_pred=RF_Model.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
cm

array([[1548,   47],
       [ 228,  177]])

# Compare to ANN model 

In [104]:
# for ANN model we need to standardize X

from sklearn.preprocessing import StandardScaler

SC=StandardScaler()
X_train=SC.fit_transform(X_train)

X_test=SC.transform(X_test)

X_train.shape

(8000, 11)

In [62]:
%timeit

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout


from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV




def built_classifier(optimizer):
    Classifier=Sequential()
    Classifier.add(Dense(output_dim=12, init='uniform', activation='relu', input_dim=11))
    Classifier.add(Dropout(p=0.05))
    Classifier.add(Dense(output_dim=10, activation='relu'))
    Classifier.add(Dropout(p=0.05))
    Classifier.add(Dense(output_dim=10, activation='relu'))
    Classifier.add(Dense(output_dim=1,init ='uniform', activation='sigmoid'))
    Classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return Classifier


Classifier=KerasClassifier(build_fn= built_classifier,epochs=100 )

parameters={'batch_size':[5,15],'optimizer':['adam','rmsprop'] }


GS=GridSearchCV(Classifier, parameters, cv=4, scoring='accuracy')

GS.fit(X_train , y_train)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=11, units=12, kernel_initializer="uniform")`
  app.launch_new_instance()
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.05)`
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=10)`
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.05)`
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=10)`
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, k

Epoch 1/100
6000/6000 [==============================] - 3s 531us/step - loss: 0.4457 - acc: 0.7985
Epoch 2/100
6000/6000 [==============================] - 2s 313us/step - loss: 0.4066 - acc: 0.8087
Epoch 3/100
6000/6000 [==============================] - 2s 311us/step - loss: 0.3877 - acc: 0.8425
Epoch 4/100
6000/6000 [==============================] - 2s 314us/step - loss: 0.3775 - acc: 0.8513
Epoch 5/100
6000/6000 [==============================] - 2s 344us/step - loss: 0.3753 - acc: 0.8505
Epoch 6/100
6000/6000 [==============================] - 2s 324us/step - loss: 0.3682 - acc: 0.8537
Epoch 7/100
6000/6000 [==============================] - 2s 313us/step - loss: 0.3704 - acc: 0.8552
Epoch 8/100
6000/6000 [==============================] - 2s 312us/step - loss: 0.3686 - acc: 0.8560
Epoch 9/100
6000/6000 [==============================] - 2s 302us/step - loss: 0.3681 - acc: 0.8568
Epoch 10/100
6000/6000 [==============================] - 2s 310us/step - loss: 0.3611 - acc: 0.8552

6000/6000 [==============================] - 2s 280us/step - loss: 0.3402 - acc: 0.8653
Epoch 83/100
6000/6000 [==============================] - 2s 277us/step - loss: 0.3364 - acc: 0.8665
Epoch 84/100
6000/6000 [==============================] - 2s 274us/step - loss: 0.3391 - acc: 0.8675
Epoch 85/100
6000/6000 [==============================] - 2s 279us/step - loss: 0.3368 - acc: 0.8653
Epoch 86/100
6000/6000 [==============================] - 2s 274us/step - loss: 0.3399 - acc: 0.8653
Epoch 87/100
6000/6000 [==============================] - 2s 276us/step - loss: 0.3377 - acc: 0.8650
Epoch 88/100
6000/6000 [==============================] - 2s 278us/step - loss: 0.3369 - acc: 0.8665
Epoch 89/100
6000/6000 [==============================] - 2s 276us/step - loss: 0.3371 - acc: 0.8692
Epoch 90/100
6000/6000 [==============================] - 2s 276us/step - loss: 0.3407 - acc: 0.8657
Epoch 91/100
6000/6000 [==============================] - 2s 280us/step - loss: 0.3362 - acc: 0.8648
Epo

6000/6000 [==============================] - 2s 285us/step - loss: 0.3370 - acc: 0.8640
Epoch 64/100
6000/6000 [==============================] - 2s 278us/step - loss: 0.3377 - acc: 0.8638
Epoch 65/100
6000/6000 [==============================] - 2s 283us/step - loss: 0.3347 - acc: 0.8663
Epoch 66/100
6000/6000 [==============================] - 2s 285us/step - loss: 0.3367 - acc: 0.8650
Epoch 67/100
6000/6000 [==============================] - 2s 280us/step - loss: 0.3321 - acc: 0.8657
Epoch 68/100
6000/6000 [==============================] - 2s 282us/step - loss: 0.3338 - acc: 0.8662
Epoch 69/100
6000/6000 [==============================] - 2s 288us/step - loss: 0.3379 - acc: 0.8670
Epoch 70/100
6000/6000 [==============================] - 2s 282us/step - loss: 0.3342 - acc: 0.8645
Epoch 71/100
6000/6000 [==============================] - 2s 279us/step - loss: 0.3353 - acc: 0.8662
Epoch 72/100
6000/6000 [==============================] - 2s 287us/step - loss: 0.3343 - acc: 0.8675
Epo

6000/6000 [==============================] - 2s 286us/step - loss: 0.3340 - acc: 0.8645
Epoch 45/100
6000/6000 [==============================] - 2s 282us/step - loss: 0.3382 - acc: 0.8623
Epoch 46/100
6000/6000 [==============================] - 2s 289us/step - loss: 0.3292 - acc: 0.8680
Epoch 47/100
6000/6000 [==============================] - 2s 292us/step - loss: 0.3333 - acc: 0.8672
Epoch 48/100
6000/6000 [==============================] - 2s 282us/step - loss: 0.3358 - acc: 0.8648
Epoch 49/100
6000/6000 [==============================] - 2s 285us/step - loss: 0.3343 - acc: 0.8658
Epoch 50/100
6000/6000 [==============================] - 2s 293us/step - loss: 0.3319 - acc: 0.8652
Epoch 51/100
6000/6000 [==============================] - 2s 286us/step - loss: 0.3328 - acc: 0.8645
Epoch 52/100
6000/6000 [==============================] - 2s 283us/step - loss: 0.3325 - acc: 0.8647
Epoch 53/100
6000/6000 [==============================] - 2s 308us/step - loss: 0.3349 - acc: 0.8647
Epo

...

6000/6000 [==============================] - 1s 94us/step - loss: 0.3348 - acc: 0.8645
Epoch 58/100
6000/6000 [==============================] - 1s 102us/step - loss: 0.3393 - acc: 0.8625
Epoch 59/100
6000/6000 [==============================] - 1s 103us/step - loss: 0.3375 - acc: 0.8643
Epoch 60/100
6000/6000 [==============================] - 1s 99us/step - loss: 0.3380 - acc: 0.8633
Epoch 61/100
6000/6000 [==============================] - 1s 105us/step - loss: 0.3353 - acc: 0.8630
Epoch 62/100
6000/6000 [==============================] - 1s 95us/step - loss: 0.3317 - acc: 0.8658
Epoch 63/100
6000/6000 [==============================] - 1s 101us/step - loss: 0.3439 - acc: 0.8622
Epoch 64/100
6000/6000 [==============================] - 1s 98us/step - loss: 0.3374 - acc: 0.8652
Epoch 65/100
6000/6000 [==============================] - 1s 97us/step - loss: 0.3377 - acc: 0.8648
Epoch 66/100
6000/6000 [==============================] - 1s 100us/step - loss: 0.3332 - acc: 0.8675
Epoch 67

6000/6000 [==============================] - 1s 99us/step - loss: 0.3429 - acc: 0.8613
Epoch 39/100
6000/6000 [==============================] - 1s 101us/step - loss: 0.3426 - acc: 0.8632
Epoch 40/100
6000/6000 [==============================] - 1s 98us/step - loss: 0.3494 - acc: 0.8598
Epoch 41/100
6000/6000 [==============================] - 1s 102us/step - loss: 0.3424 - acc: 0.8590
Epoch 42/100
6000/6000 [==============================] - 1s 98us/step - loss: 0.3480 - acc: 0.8622: 0s - loss: 0.3491 - acc: 0.86
Epoch 43/100
6000/6000 [==============================] - 1s 101us/step - loss: 0.3491 - acc: 0.8600
Epoch 44/100
6000/6000 [==============================] - 1s 101us/step - loss: 0.3414 - acc: 0.8640
Epoch 45/100
6000/6000 [==============================] - 1s 96us/step - loss: 0.3422 - acc: 0.8610
Epoch 46/100
6000/6000 [==============================] - 1s 102us/step - loss: 0.3430 - acc: 0.8588
Epoch 47/100
6000/6000 [==============================] - 1s 96us/step - loss

8000/8000 [==============================] - 3s 337us/step - loss: 0.3389 - acc: 0.8653
Epoch 20/100
8000/8000 [==============================] - 3s 333us/step - loss: 0.3421 - acc: 0.8628
Epoch 21/100
8000/8000 [==============================] - 3s 327us/step - loss: 0.3421 - acc: 0.8605
Epoch 22/100
8000/8000 [==============================] - 3s 331us/step - loss: 0.3387 - acc: 0.8655
Epoch 23/100
8000/8000 [==============================] - 3s 332us/step - loss: 0.3368 - acc: 0.8649
Epoch 24/100
8000/8000 [==============================] - 3s 335us/step - loss: 0.3357 - acc: 0.8643
Epoch 25/100
8000/8000 [==============================] - 3s 334us/step - loss: 0.3371 - acc: 0.8669
Epoch 26/100
8000/8000 [==============================] - 3s 331us/step - loss: 0.3372 - acc: 0.8628
Epoch 27/100
8000/8000 [==============================] - 3s 330us/step - loss: 0.3354 - acc: 0.8633
Epoch 28/100
8000/8000 [==============================] - 3s 334us/step - loss: 0.3381 - acc: 0.8624
Epo

8000/8000 [==============================] - 3s 333us/step - loss: 0.3294 - acc: 0.8660


GridSearchCV(cv=4, error_score='raise',
       estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x13ac70dd8>,
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'batch_size': [5, 15], 'optimizer': ['adam', 'rmsprop']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [64]:
GS.best_params_

{'batch_size': 5, 'optimizer': 'adam'}

In [65]:
GS.best_score_

0.8635